In [1]:
from My_Cspro_function import *
from value_sets_output import VALUE_SETS

In [10]:
download_ftp_files()

❌ Erreur : 


In [4]:
Unzip_All_Files()

Successfully extracted Data_Zip\DATA101.zip to extracted_data
Successfully extracted Data_Zip\DATA103.zip to extracted_data
Successfully extracted Data_Zip\DATA104.zip to extracted_data


In [17]:
import os
import pandas as pd

def extraire_et_agreger_csdb(rep_csdb):
    """
    Parcourt tous les fichiers .csdb du répertoire,
    extrait les tables cheque_rec, level, rejets, synthe,
    applique les merges, et renvoie deux dataframes agrégés :
    - df_rejets_final
    - df_synthe_final
    """

    liste_rejets = []
    liste_synthe = []
    liste_tabr=[]
    columns_label_rejet={
    "numfact":"Numéro de facture",
    "numch":"Numéro de chèque",
    'statinc':"Statut intial chèque",
    'noms':"Nom du répondant",
    'cat':"Catégorie",
    'typpres': "Type de prestation",
    'mfact':"Montant facturé",
    'mvaldmc':"Montant validé par le MC",
    'mvaldccm':"Montant validé par le Contr M",
    'mvaldcm':"Montant rejeté par le CM",
    'statch':"statut du chèque",
    'motif':"Motif de rejet_réhabilitation",
    'mfosa':"FOSA",
    'ms0q03':"NOM AIRE DE SANTE",
    's0q03':"Aire de santé",
    'ms0q02':"NOM DISTRICT",
    's0q01':"Région",
    's0q02':"District",
    's0q3':"Statut FOSA",
    's0q05':"Date collecte",
    'cenq':"Agent enqueteur",   
}

    colums_label_synthese={
        'numf':"Numéro ligne",
        'type':"Type de prestation",
        "statut":"Prestation",
        'prest01':"TARIF UNITAIRE PRESTATION",
        'prest02':"NOMBRE TOTAL DE FACTURE RECU",
        'prest03':"MONTANT TOTAL FACTURE RECU",
        'prest04':"NOMBRE TOTAL DE FACTURE VALIDE MC",
        'prest05':"MONTANT TOTAL DE FACTURE NON VALIDE MC",
        'prest06':"MONTANT TOTAL DE FACTURE  VALIDE MC",
        'mfosa':"FOSA",
        's0q03':"Aire de santé",
        's0q01':"Région",
        's0q02':"District",
        's0q3':"Statut FOSA",
        's0q05':"Date collecte",
        'ms0q03':"NOM AIRE DE SANTE",
        'ms0q02':"NOM DISTRICT",
        'cenq':"Agent enqueteur",
    }
    
    colums_label_tabr={
        'num':"Nomre de chèque à saisir",
        'mfosa':"FOSA",
        's0q03':"Aire de santé",
        's0q01':"Région",
        's0q02':"District",
        's0q3':"Statut FOSA",
        's0q05':"Date collecte",
        'cenq':"Agent enqueteur",
        'labmois':"Mois",
        'ms0q03':"NOM AIRE DE SANTE",
        'ms0q02':"NOM DISTRICT",
    }
    # Parcours de tous les fichiers .csdb
    for f in os.listdir(rep_csdb):
        if f.lower().endswith(".csdb"):
            chemin = os.path.join(rep_csdb, f)
            print(f"📂 Traitement de : {f}")

            try:
                # Lecture des tables
                cheque = lire_csdb(chemin, table_name="cheque_rec")
                lev = lire_csdb(chemin, table_name="level")
                rejet = lire_csdb(chemin, table_name="rejets")
                synthe = lire_csdb(chemin, table_name="synthe")
                tabr=lire_csdb(chemin, table_name="tabr")

                
                tabr=tabr.merge(cheque, on="level-1-id", how="left")
                liste_tabr.append(tabr)
                # Merge cheque + level
                cheque = cheque.merge(lev, on="level-1-id", how="left")

                # Merge rejet + cheque
                rejet = rejet.merge(cheque, on="level-1-id", how="left")
                liste_rejets.append(rejet)

                # Merge synthe + cheque
                synthe = synthe.merge(cheque, on="level-1-id", how="left")
                liste_synthe.append(synthe)

            except Exception as e:
                print(f"❌ Erreur sur le fichier {f} : {e}")
                continue

    # Agrégation finale
    if liste_rejets:
        df_rejets_final = pd.concat(liste_rejets, ignore_index=True)
    else:
        df_rejets_final = pd.DataFrame()

    if liste_synthe:
        df_synthe_final = pd.concat(liste_synthe, ignore_index=True)
    else:
        df_synthe_final = pd.DataFrame()
    if liste_tabr:
        tabr_final=pd.concat(liste_tabr, ignore_index=True)
    else:
        tabr_final=pd.DataFrame()
        
    
    # Sélectionner toutes les colonnes numériques
    #num_cols_rejet = df_rejets_final.select_dtypes(include=["float", "int"]).columns
    #df_rejets_final[num_cols_rejet]=df_rejets_final[num_cols_rejet].astype("Int64")
    
    #num_cols_synthese = df_synthe_final.select_dtypes(include=["float", "int"]).columns
    #df_synthe_final[num_cols_synthese]=df_synthe_final[num_cols_synthese].astype("Int64")
    
    #Maping
    df_rejets_final['statinc']=df_rejets_final['statinc'].replace(VALUE_SETS['statinc'])
    df_rejets_final['cat']=df_rejets_final['cat'].replace(VALUE_SETS['cat'])
    df_rejets_final['typpres']=df_rejets_final['typpres'].replace(VALUE_SETS['typpres'])
    df_rejets_final['statch']=df_rejets_final['statch'].replace(VALUE_SETS['statch'])
    df_rejets_final['motif']=df_rejets_final['motif'].replace(VALUE_SETS['motif'])
    df_rejets_final['s0q03']=df_rejets_final['s0q03'].replace(VALUE_SETS['s0q03'])
    df_rejets_final['s0q01']=df_rejets_final['s0q01'].replace(VALUE_SETS['s0q01'])
    df_rejets_final['s0q02']=df_rejets_final['s0q02'].replace(VALUE_SETS['s0q02'])
    df_rejets_final['s0q3']=df_rejets_final['s0q3'].replace(VALUE_SETS['s0q3'])
    df_rejets_final['mois']=df_rejets_final['mois'].replace(VALUE_SETS['mois'])
    
    df_synthe_final['type']=df_synthe_final['type'].replace(VALUE_SETS['type'])
    df_synthe_final['statut']=df_synthe_final['statut'].replace(VALUE_SETS['typpres'])
    df_synthe_final['s0q03']=df_synthe_final['s0q03'].replace(VALUE_SETS['s0q03'])
    df_synthe_final['s0q01']=df_synthe_final['s0q01'].replace(VALUE_SETS['s0q01'])
    df_synthe_final['s0q02']=df_synthe_final['s0q02'].replace(VALUE_SETS['s0q02'])

    tabr_final['s0q03']=tabr_final['s0q03'].replace(VALUE_SETS['s0q03'])
    tabr_final['s0q01']=tabr_final['s0q01'].replace(VALUE_SETS['s0q01'])
    tabr_final['s0q02']=tabr_final['s0q02'].replace(VALUE_SETS['s0q02'])
    tabr_final['s0q3']=tabr_final['s0q3'].replace(VALUE_SETS['s0q3'])
        
    #Rétension des colonne utiles et renomage
    df_rejets_final = df_rejets_final.drop(columns=["cheque_rec-id", "case-id", "numfact_1","labmois","action1"])
    df_rejets_final=df_rejets_final.rename(columns=columns_label_rejet)
    
    df_synthe_final = df_synthe_final.drop(columns=["synthe-id", "action", "cheque_rec-id","case-id","nlg","labmois"])
    df_synthe_final=df_synthe_final.rename(columns=colums_label_synthese)
    
    tabr_final = tabr_final.drop(columns=["tabr-id", "ftab1", "cheque_rec-id","nlg"])
    tabr_final=tabr_final.rename(columns=colums_label_tabr)
    
    fichier_data="Data_collected.xlsx"
    
    with pd.ExcelWriter(fichier_data, engine="xlsxwriter") as writer:
        df_rejets_final.to_excel(writer, sheet_name="Rejet", index=False)
        df_synthe_final.to_excel(writer, sheet_name="Synthese", index=False)
        tabr_final.to_excel(writer, sheet_name="Initial", index=False)
    
    
    return df_rejets_final, df_synthe_final, tabr_final


In [18]:
df_rejets, df_synthe, df_initial = extraire_et_agreger_csdb("extracted_data")

📂 Traitement de : S10120120102.csdb
📂 Traitement de : S10120120104.csdb
📂 Traitement de : S10120120106.csdb
📂 Traitement de : S10120120108.csdb
📂 Traitement de : S10320120105.csdb
📂 Traitement de : S10320120107.csdb
📂 Traitement de : S10320220214.csdb
📂 Traitement de : S10320220215.csdb
📂 Traitement de : S10320220217.csdb
📂 Traitement de : S10420120103.csdb
📂 Traitement de : S10420120105.csdb
📂 Traitement de : S10420120107.csdb
📂 Traitement de : S10420220214.csdb
📂 Traitement de : S10420220215.csdb
📂 Traitement de : S10420220217.csdb


C:\Users\LK\AppData\Local\Temp\ipykernel_5784\1426059330.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_rejets_final = pd.concat(liste_rejets, ignore_index=True)
C:\Users\LK\AppData\Local\Temp\ipykernel_5784\1426059330.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tabr_final=pd.concat(liste_tabr, ignore_index=True)


In [12]:
df_initial

,level-1-id,Nomre de chèque à saisir,FOSA,Aire de santé,NOM AIRE DE SANTE,Région,District,NOM DISTRICT,Mois,Statut FOSA,Agent enqueteur,Date collecte
0,1,NaN,CS MISSION CHRETIENNE NKOEMVONE ...,AS NYABESSANG,BIBA1 ...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025
1,2,NaN,CS MISSION CHRETIENNE NKOEMVONE ...,AS NYABESSANG,BIBA1 ...,Sud,DS AMBAM,EBOLOWA ...,Fervrier ...,SONUB,101,14112025
2,3,0.0,CS MISSION CHRETIENNE NKOEMVONE ...,AS NYABESSANG,BIBA1 ...,Sud,DS AMBAM,EBOLOWA ...,Juin ...,SONUB,101,15112025
3,4,1.0,CS MISSION CHRETIENNE NKOEMVONE ...,AS NYABESSANG,BIBA1 ...,Sud,DS AMBAM,EBOLOWA ...,Mai ...,SONUB,101,15112025
4,1,0.0,HOPITAL REGIONAL EBOLOWA ...,AS AMBAM URBAIN,ENONGAL ...,Sud,DS AMBAM,EBOLOWA ...,Mars ...,SONUC,101,18112025
...,...,...,...,...,...,...,...,...,...,...,...,...
61,2,0.0,CSI MESSOK ...,20215,MESSOK ...,Sud,DS DJOUM,MEYOMESSALA ...,Juillet ...,SONUB,104,16112025
62,3,7.0,CSI MESSOK ...,20215,MESSOK ...,Sud,DS DJOUM,MEYOMESSALA ...,Mars ...,SONUB,104,16112025
63,4,0.0,CSI MESSOK ...,20215,MESSOK ...,Sud,DS DJOUM,MEYOMESSALA ...,Juin ...,SONUB,104,16112025
64,1,0.0,HÔPITAL DISTRICT MEYOMESSALA ...,20217,MEYOMESSALA ...,Sud,DS DJOUM,MEYOMESSALA ...,Janvier ...,SONUC,104,16112025


,tabr-id,level-1-id,ftab1,num,cheque_rec-id,mfosa,s0q03,ms0q03,s0q01,s0q02,ms0q02,labmois,s0q3,cenq,s0q05,nlg
0,1,1,1,NaN,1,CS MISSION CHRETIENNE NKOEMVONE ...,20102,BIBA1 ...,2,201,EBOLOWA ...,Janvier ...,1,101,14112025,14
1,2,2,1,NaN,2,CS MISSION CHRETIENNE NKOEMVONE ...,20102,BIBA1 ...,2,201,EBOLOWA ...,Fervrier ...,1,101,14112025,15
2,3,3,1,0.0,3,CS MISSION CHRETIENNE NKOEMVONE ...,20102,BIBA1 ...,2,201,EBOLOWA ...,Juin ...,1,101,15112025,19
3,4,4,1,1.0,4,CS MISSION CHRETIENNE NKOEMVONE ...,20102,BIBA1 ...,2,201,EBOLOWA ...,Mai ...,1,101,15112025,18
4,1,1,1,0.0,1,HOPITAL REGIONAL EBOLOWA ...,20104,ENONGAL ...,2,201,EBOLOWA ...,Mars ...,2,101,18112025,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2,2,1,0.0,2,CSI MESSOK ...,20215,MESSOK ...,2,202,MEYOMESSALA ...,Juillet ...,1,104,16112025,10
62,3,3,1,7.0,3,CSI MESSOK ...,20215,MESSOK ...,2,202,MEYOMESSALA ...,Mars ...,1,104,16112025,10
63,4,4,1,0.0,4,CSI MESSOK ...,20215,MESSOK ...,2,202,MEYOMESSALA ...,Juin ...,1,104,16112025,9
64,1,1,1,0.0,1,HÔPITAL DISTRICT MEYOMESSALA ...,20217,MEYOMESSALA ...,2,202,MEYOMESSALA ...,Janvier ...,2,104,16112025,2


In [7]:
df_rejets

,rejets-id,level-1-id,occ,nrejet,Numéro de facture,Numéro de chèque,Statut intial chèque,Nom du répondant,Catégorie,Type de prestation,...,Région,District,NOM DISTRICT,Statut FOSA,Agent enqueteur,Date collecte,nlg,cheque_id,annee,mois
0,1,4,1,1,360448.0,504360.0,Chèque rejeté selon le Medecin conseil,Akamba Essono Leonella ...,Prestations forfaitaires,CPN1,...,Sud,DS AMBAM,EBOLOWA ...,SONUB,101,15112025,18,2010202,2024,Mai
1,1,9,1,1,516048.0,506227.0,Chèque validé selon le Medecin conseil,minko assam douda ...,Prestations forfaitaires,ECHO,...,Sud,DS AMBAM,EBOLOWA ...,SONUB,101,14112025,15,2010608,2024,Avril
2,2,9,2,2,686505.0,506224.0,Chèque validé selon le Medecin conseil,angue rose ...,Prestations non médicales,401.0,...,Sud,DS AMBAM,EBOLOWA ...,SONUB,101,14112025,15,2010608,2024,Avril
3,3,9,3,3,770115.0,506212.0,Chèque validé selon le Medecin conseil,eda ingrid leslie ...,Prestations non médicales,401.0,...,Sud,DS AMBAM,EBOLOWA ...,SONUB,101,14112025,15,2010608,2024,Avril
4,4,9,4,4,516048.0,506227.0,Chèque validé selon le Medecin conseil,minko assako ...,Prestations non médicales,401.0,...,Sud,DS AMBAM,EBOLOWA ...,SONUB,101,14112025,15,2010608,2024,Avril
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,3,3,3,3,455.0,508455.0,Chèque validé selon le Medecin conseil,MENGUE ABOMO ...,Prestations forfaitaires,ECHO,...,Sud,DS DJOUM,MEYOMESSALA ...,SONUB,104,16112025,10,2021518,2024,Mars
220,4,3,4,4,456.0,508456.0,Chèque validé selon le Medecin conseil,NKOUMOU JOSEPHINE ...,Prestations forfaitaires,ECHO,...,Sud,DS DJOUM,MEYOMESSALA ...,SONUB,104,16112025,10,2021518,2024,Mars
221,5,3,5,5,461.0,508461.0,Chèque validé selon le Medecin conseil,MBENGONO ETOUNGOU ...,Prestations forfaitaires,ECHO,...,Sud,DS DJOUM,MEYOMESSALA ...,SONUB,104,16112025,10,2021518,2024,Mars
222,6,3,6,6,479.0,508479.0,Chèque validé selon le Medecin conseil,MBALLA ZE BLANCHE ...,Prestations forfaitaires,CPN1,...,Sud,DS DJOUM,MEYOMESSALA ...,SONUB,104,16112025,10,2021518,2024,Mars


In [20]:
df_rejets.columns

Index(['rejets-id', 'level-1-id', 'occ', 'nrejet', 'numfact', 'numch',
       'statinc', 'noms', 'cat', 'typpres', 'typpresx', 'mfact', 'mvaldmc',
       'mvaldccm', 'mvaldcm', 'statch', 'motif', 'motifx', 'action1',
       'cheque_rec-id', 'mfosa', 's0q03', 'ms0q03', 's0q01', 's0q02', 'ms0q02',
       'labmois', 's0q3', 'cenq', 's0q05', 'nlg', 'case-id', 'cheque_id',
       'annee', 'mois', 'numfact_1'],
      dtype='object')

In [18]:
df_synthe = df_synthe.drop(columns=["synthe-id", "action", "cheque_rec-id","case-id","nlg"])
df_synthe

,level-1-id,occ,numf,type,statut,statutx,prest01,prest02,prest03,prest04,...,s0q01,s0q02,ms0q02,labmois,s0q3,cenq,s0q05,cheque_id,annee,mois
0,1,1,1,1.0,101.0,...,7000.0,24.0,168000.0,24.0,...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025,2010202,2024,Janvier
1,1,2,2,1.0,102.0,...,2500.0,31.0,77500.0,31.0,...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025,2010202,2024,Janvier
2,1,3,3,1.0,103.0,...,2500.0,31.0,77500.0,31.0,...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025,2010202,2024,Janvier
3,1,4,4,1.0,104.0,...,2500.0,11.0,27500.0,11.0,...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025,2010202,2024,Janvier
4,1,5,5,1.0,107.0,...,225.0,6.0,1350.0,6.0,...,Sud,DS AMBAM,EBOLOWA ...,Janvier ...,SONUB,101,14112025,2010202,2024,Janvier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,4,9,9,1.0,106.0,...,225.0,1.0,225.0,1.0,...,Sud,DS DJOUM,MEYOMESSALA ...,Juin ...,SONUB,104,16112025,2021518,2024,Juin
991,1,1,1,3.0,318.0,...,92600.0,6.0,421270.0,6.0,...,Sud,DS DJOUM,MEYOMESSALA ...,Janvier ...,SONUC,104,16112025,2021720,2024,Janvier
992,1,2,2,3.0,314.0,...,7500.0,1.0,7405.0,1.0,...,Sud,DS DJOUM,MEYOMESSALA ...,Janvier ...,SONUC,104,16112025,2021720,2024,Janvier
993,2,1,1,3.0,318.0,...,92600.0,6.0,421270.0,6.0,...,Sud,DS DJOUM,MEYOMESSALA ...,Avril ...,SONUC,104,16112025,2021720,2024,Avril


In [19]:
df_synthe.columns

Index(['level-1-id', 'occ', 'numf', 'type', 'statut', 'statutx', 'prest01',
       'prest02', 'prest03', 'prest04', 'prest05', 'prest06', 'mfosa', 's0q03',
       'ms0q03', 's0q01', 's0q02', 'ms0q02', 'labmois', 's0q3', 'cenq',
       's0q05', 'cheque_id', 'annee', 'mois'],
      dtype='object')

In [9]:
from ftplib import FTP, all_errors

# Informations FTP
ftp_host = "217.112.80.251"
ftp_path = "/CHEQUE2025/ENQUETE/DATA"
ftp_user = "user_ins"
ftp_pass = "123456@"

ftp = None
liste_brute = []

# Essayer d'abord le mode passif (recommandé), puis basculer en actif si besoin.
# Augmenter le timeout pour éviter les TimeoutError sur des connexions lentes.
for passive in (True, False):
    try:
        ftp = FTP()
        ftp.set_pasv(passive)
        timeout = 60 if passive else 120
        ftp.connect(ftp_host, 21, timeout=timeout)
        ftp.login(ftp_user, ftp_pass)
        print(f"Connexion réussie ! (passive={passive}, timeout={timeout})")
        ftp.cwd(ftp_path)
        # Récupérer la liste brute via LIST
        liste_brute.clear()
        ftp.retrlines("LIST", liste_brute.append)

        # Construire une liste contenant uniquement les noms des dossiers
        liste_dossiers = [ligne.split()[-1] for ligne in liste_brute if ligne.startswith("d")]
        print(f"Dossiers trouvés : {liste_dossiers}")

        break
    except (TimeoutError, ConnectionError) as e:
        print(f"Timeout/Connection error with passive={passive}: {e!r}")
        if ftp:
            try:
                ftp.close()
            except Exception:
                pass
        ftp = None
    except all_errors as e:
        print(f"FTP error with passive={passive}: {e!r}")
        if ftp:
            try:
                ftp.close()
            except Exception:
                pass
        ftp = None

if not liste_brute:
    raise TimeoutError("Impossible d'obtenir la liste du serveur FTP. Vérifiez la connectivité, le pare-feu, le mode passif/actif et les identifiants.")

print("\n=== Contenu du répertoire ===\n")

fichiers_dossiers = []

for ligne in liste_brute:
    # Exemple de ligne :
    # drwxr-xr-x   2 user group      4096 Jan 01 12:00 dossier
    # -rw-r--r--   1 user group     10240 Jan 01 12:00 fichier.txt

    parties = ligne.split()
    nom = parties[-1]  # Le nom est toujours le dernier élément

    if ligne.startswith("d"):
        type_objet = "Dossier"
    else:
        type_objet = "Fichier"

    fichiers_dossiers.append((type_objet, nom))

    print(f"{type_objet:10}  ->  {nom}")

if ftp:
    try:
        ftp.quit()
    except Exception:
        try:
            ftp.close()
        except Exception:
            pass


Connexion réussie ! (passive=True, timeout=60)
Dossiers trouvés : ['101', '103', '104']

=== Contenu du répertoire ===

Dossier     ->  101
Dossier     ->  103
Dossier     ->  104
